In [ ]:
from Tbx_Sample import *

In [ ]:
class Participant:
    def __init__(self, labels, stochastic_params, mnist_population, available_indexes):
        """
        Initialize the Participant with labels and their stochastic process parameters.
        
        labels: List of labels to sample from (e.g., [0, 1, 2, 3, ...])
        stochastic_params: A list of dictionaries containing the stochastic process parameters for each label
        mnist_population: The indexed MNIST population
        available_indexes: The available indexes for sampling from the MNIST population
        """
        self.labels = labels
        self.stochastic_params = stochastic_params
        self.mnist_population = mnist_population
        self.available_indexes = available_indexes
        self.Samples = self._generate_Samples()
    
    def _generate_Samples(self):
        """
        Generate samples for each label based on their respective stochastic process parameters.
        """
        samples = {}
        for label, params in zip(self.labels, self.stochastic_params):
            try:
                # Create the stochastic process for this label
                stochastic_process = StochasticProcess(
                    initial_population=params['initial_population'],
                    steps=params['steps'],
                    process_type=params['process_type'],
                    combine_with=params['combine_with'],
                    rate=params['rate'],
                    angular_coef=params['angular_coef'],
                    period=params['period'],
                    frequency=params['frequency']
                )
                # Create a sample instance for this label
                sample = Sample(
                    label=label,
                    stochastic_process=stochastic_process,
                    mnist_population=self.mnist_population,
                    available_indexes=self.available_indexes
                )
                # Store the samples for this label
                samples[label] = sample
            except KeyError as e:
                print(f"Missing parameter {e} for label {label}")
            except Exception as e:
                print(f"An error occurred for label {label}: {e}")
        return samples
    
    def get_sample_label(self, label, time_step):
        """
        Get the samples for a specific label at a given time step.
        
        label: The label for which samples are needed (e.g., 3)
        time_step: The time step at which to fetch the samples
        """
        try:
            sample = self.Samples[label]
            return sample.get_samples_at(time_step)
        except KeyError:
            print(f"Sample for label {label} not found.")
        except Exception as e:
            print(f"An error occurred: {e}")

    def report_data(self):
        aux_data = {}
        for label in self.labels:
            aux_data[label] = self.Samples[label].get_process_data()
        
        df = pd.DataFrame.from_dict(aux_data, orient='index')
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'id'}, inplace=True)
        df.columns = ['id'] + [f'value_{i+1}' for i in range(df.shape[1] - 1)]

        return df

In [ ]:
import itertools
import random

def generate_limited_values(start, stop, step, max_values=None):
    values = list(frange(start, stop, step))
    if max_values and len(values) > max_values:
        values = random.sample(values, max_values)
    return values

def frange(start, stop, step):
    while start < stop:
        yield round(start, 10)  # Avoid floating-point precision errors
        start += step

def generate_stochastic_combinations(init_pop=200, init_steps=20, init_freq=None, init_period=None, max_combinations=None):
    if init_freq is None:
        init_freq = generate_limited_values(0.1 * init_pop, 0.5 * init_pop, 0.05 * init_pop, max_combinations)

    if init_period is None:
        init_period = generate_limited_values(0, 0.5, 0.07, max_combinations)

    # Define ranges for rate and angular coefficient
    init_rate_range = generate_limited_values(-0.15, 0.15, 0.05, max_combinations)
    init_angular_range = generate_limited_values(-4, 4, 0.5, max_combinations)

    # Define possible values for 'combine_with' and 'process_type'
    combine_with_values = ['exponential', 'sine', 'exp_decay']
    process_type_values = ['combined']

    # Generate valid combinations of 'combine_with'
    combine_with_combinations = [
        comb for r in range(1, 3)  # Generate combinations of size 1 and 2
        for comb in itertools.combinations(combine_with_values, r)
        if not ('exponential' in comb and 'exp_decay' in comb)
    ]

    # Generate all combinations
    param_combinations = list(itertools.product(init_rate_range, init_angular_range, combine_with_combinations, process_type_values))

    # If max_combinations is specified, sample from it
    if max_combinations and len(param_combinations) > max_combinations:
        param_combinations = random.sample(param_combinations, max_combinations)

    # Generate the parameter dictionaries
    parameter_dicts = []
    for rate, angular, combine, process in param_combinations:
        param_dict = {
            'initial_population': init_pop,
            'steps': init_steps,
            'process_type': process,
            'combine_with': list(combine),
            'rate': rate,
            'angular_coef': angular,
            'period': random.choice(init_period) if isinstance(init_period, list) else init_period,
            'frequency': random.choice(init_freq) if isinstance(init_freq, list) else init_freq,
        }
        parameter_dicts.append(param_dict)

    return parameter_dicts